In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from tqdm import tqdm
import numpy as np

/home/ubuntu/anaconda3/envs/content_understanding/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reward_name = "OpenAssistant/reward-model-deberta-v3-large-v2"
rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(reward_name), AutoTokenizer.from_pretrained(reward_name, model_max_length=1700)
rank_model = rank_model.to('cuda')

2023-09-12 16:02:14.363086: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
base_dir = 'data/openai_summarize_from_feedback'
original = pd.read_csv(f"{base_dir}/english_original.csv")
chinese = pd.read_csv(f"{base_dir}/english_to_chinese.csv")
english = pd.read_csv(f"{base_dir}/english_to_chinese_to_english.csv")

In [4]:
original.shape, english.shape, chinese.shape

((83802, 3), (83802, 3), (83802, 3))

In [5]:
def get_score(prompt, response):
    inputs = tokenizer(prompt, response, return_tensors='pt', truncation=True)
    inputs = inputs.to('cuda')
    return rank_model(**inputs).logits[0].cpu().detach().item()

In [ ]:
def get_batch_score(prompts, responses):
    inputs = tokenizer(prompts, responses, padding=True, return_tensors='pt').to('cuda')
    return rank_model(**inputs).logits.flatten().tolist()

In [12]:
original_prompt_chinese_response = pd.DataFrame({
    'prompt' :original['prompt'],
    'chosen' : chinese['chosen'],
    'rejected' : chinese['rejected']
})
chinese_prompt_original_response = pd.DataFrame({
    'prompt' :chinese['prompt'],
    'chosen' : original['chosen'],
    'rejected' : original['rejected']
})

In [15]:
dataset_map = {
    #'original' : original,
    #'chinese' : chinese,
    #'english' : english,
    'original_prompt_chinese_response' : original_prompt_chinese_response,
    #'chinese_prompt_original_response' : chinese_prompt_original_response
}

In [16]:
for k,v in dataset_map.items():
    dataset_map[k] = v.fillna('')

In [17]:
all_scores = {}
for name, dataframe in dataset_map.items():
    chosen_scores = []
    reject_scores = []
    existing = {}
    for i, row in tqdm(dataframe.iterrows(), total=len(original)):
        if (row.prompt, row.chosen) not in existing:
            existing[(row.prompt, row.chosen)] = get_score(row.prompt, row.chosen)
        chosen_scores.append(existing[(row.prompt, row.chosen)])

        if (row.prompt, row.rejected) not in existing:
            existing[(row.prompt, row.rejected)] = get_score(row.prompt, row.rejected)
        reject_scores.append(existing[(row.prompt, row.rejected)])

    all_scores[name] = {'chosen' : chosen_scores, 'rejected' : reject_scores}

  1%|          | 1029/83802 [00:23<47:31, 29.02it/s]

In [ ]:
for 

pd.DataFrame(all_scores['english']).to_csv('model_scores/deberta_v3_large/english_to_chinese_to_english_scores.csv', index=False)